# 식신로드, 역대 서울지역 '만점 식당' 20선
- 음식점 이름, 전화번호, 주소 크롤링하여 리스트에 저장

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import re
import requests

In [2]:
chrome_driver = 'D:/bigdata/Jupyter_python/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.implicitly_wait(5) # 응답의 시간 지연
url = 'https://www.wikitree.co.kr/articles/217101'
# chrome driver로 해당 페이지가 물리적으로 open
driver.get(url)

# 소스 가져오기
src = driver.page_source

# 소스 BeautifulSoup 객체로 변환
soup = BeautifulSoup(src)

comment_area = soup.select('div#wikicon div')

# chrome driver 사용 후, close 함수로 종료
# driver.close()

In [3]:
name = []
strong = soup.select('div#wikicon strong')
# print(strong)
for item in strong:
    if len(item.text) > 1:
        item = item.text.split('회 ')
#         print(item)
        if len(item) > 1:
            item = item[1]
            name.append(item)
print(len(name))
print(name)

20
['다성일식', '봉산집', '창고43', '돕감자탕', '대보명가', '해뜨는집', '아이 해브어 드림', '아현동 간장게장', '왕소금구이', '라틀리에 모니크', '비스트로 딩고', '줄리에뜨', '충주집', '영화루', '일품헌', '립스테이크', '오가와', '까사디노아', '충무로 주꾸미 불고기', '애성회관']


In [4]:
p = soup.select('div#wikicon p')
address = []
tel = []
content = []
del(p[0])
del(p[0])
# print(p)
for i in p:
    if '서울시' in i.text or '서울' in i.text or '종로구' in i.text:
        address.append(i.text.replace('\xa0', ' '))
    if '02' in i.text:
        tel.append(i.text.replace('\xa0', ' '))
# print(len(address))
# print(address)
print(len(tel))
print(tel)

20
['02-338-8951', '02-793-5022', '02-786-5959', '02-499-2838', '02-907-6998', '02-764-6354', '02-3453-7696', '02-312-7530', '02-545-6844', '02-549-9210', '02-518-7866', '02-535-4002', '02-923-1068', '02-738-1218', '02-574-7117', '02-379-2581', '02-735-1001', '02-3142-1108', '02-2279-0803', '02-352-0303 ']


In [55]:
result=[]
for i in range(len(name)):
    result.append((name[i], tel[i], address[i]))
print(result)

[('다성일식', '02-338-8951', '서울시 서대문구 창천동 72-36'), ('봉산집', '02-793-5022', '서울시 용산구 용산동3가 1-21'), ('창고43', '02-786-5959', '서울시 영등포구 여의도동 44-37 아일렉스타워15층(스카이점)'), ('돕감자탕', '02-499-2838', '서울시 광진구 화양동 9-22'), ('대보명가', '02-907-6998', '서울시 강북구 수유동 563-14'), ('해뜨는집', '02-764-6354', '서울시 성북구 동소문동 1가 62'), ('아이 해브어 드림', '02-3453-7696', '서울시 강남구 역삼동 821 이즈타워 B3'), ('아현동 간장게장', '02-312-7530', '서울시 마포구 아현동  282-1'), ('왕소금구이', '02-545-6844', '서울시 강남구 논현동 98-12'), ('라틀리에 모니크', '02-549-9210', '서울시 강남구 청담동 80-21'), ('비스트로 딩고', '02-518-7866', '서울시 강남구 신사동 526-11'), ('줄리에뜨', '02-535-4002', '서울시 서초구 반포4동 82-22'), ('충주집', '02-923-1068', '서울시 동대문구 제기동 138-2'), ('영화루', '02-738-1218', '서울시 종로구 누하동 25-1'), ('일품헌', '02-574-7117', '서울시 서초구 양재동 1-28'), ('립스테이크', '02-379-2581', '서울시 종로구 부암동 208-37'), ('오가와', '02-735-1001', '종로구 당주동 5 로얄빌딩 지하 1층'), ('까사디노아', '02-3142-1108', '서울 마포구 연남동 257-8 '), ('충무로 주꾸미 불고기', '02-2279-0803', '서울 중구 필동 1가 3-20'), ('애성회관', '02-352-0303 ', '서울 중구 북창동 93-36')]


# 정규표현식 사용

In [6]:
url = 'https://www.wikitree.co.kr/articles/217101'
response = requests.get(url)

In [8]:
resp = response.text.replace('&nbsp;',' ') # 공백처리
soup = BeautifulSoup(response.text)
#soup

In [10]:
raw_data = soup.select_one('div#wikicon')
#raw_data

## 음식점이름 가져오기

In [18]:
title_all = raw_data.select('div strong')
first_title_list = []
for title_one in title_all:
    title_text = title_one.text
    if title_text:
        if title_text.strip() == '14.' or title_text.strip() == '20.':
            continue
        else:
            first_title_list.append(title_text)

In [20]:
#first_title_list

In [27]:
second_title_list = []
for title_one in first_title_list:
    second_title_list.append(title_one.split('회 ')[1])

In [30]:
#second_title_list
title_list = second_title_list

## 전화번호 가져오기

In [37]:
# 전화번호 찾기 1 실패
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}') # 정규표현식 패턴 제작
tel_all = raw_data.select('div ~ p')
tel_list = []
for item in tel_all:
    str_item = str(item) # findall()이 문자열을 찾아주기 때문에 문자열로 변환
    info = tel_p.findall(str_item)
    if info:
        tel_list.append(info)
len(tel_list)

19

In [38]:
# 전화번호 찾기 2 성공
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}') # 정규표현식 패턴 제작
tel_all = raw_data.select('p')
tel_list = []
for item in tel_all:
    str_item = str(item) # findall()이 문자열을 찾아주기 때문에 문자열로 변환
    info = tel_p.findall(str_item)
    if info:
        tel_list.append(info)
len(tel_list)

20

In [44]:
# 전화번호 찾기 3 성공
tel_p = re.compile('\d{2,3}-\d{3,4}-\d{4}') # 정규표현식 패턴 제작
tel_data = raw_data.find_all(string = tel_p) # string 속성에 패턴 넣기
tel_list = []
for tel_one in tel_data:
    tel_list.append(tel_one.strip())

In [46]:
#tel_list

## 주소 가져오기

In [51]:
# 주소 찾기
address_p = re.compile('^.*구\s.*동') # 정규표현식 패턴 제작
address_data = raw_data.find_all(string = address_p) # string 속성에 패턴 넣기
address_list = []
for address_one in address_data:
    address_list.append(address_one.strip().replace('\xa0',' '))

In [52]:
address_list

['서울시 서대문구 창천동 72-36',
 '서울시 용산구 용산동3가 1-21',
 '서울시 영등포구 여의도동 44-37 아일렉스타워15층(스카이점)',
 '서울시 광진구 화양동 9-22',
 '서울시 강북구 수유동 563-14',
 '서울시 성북구 동소문동 1가 62',
 '서울시 강남구 역삼동 821 이즈타워 B3',
 '서울시 마포구 아현동  282-1',
 '서울시 강남구 논현동 98-12',
 '서울시 강남구 청담동 80-21',
 '서울시 강남구 신사동 526-11',
 '서울시 서초구 반포4동 82-22',
 '서울시 동대문구 제기동 138-2',
 '서울시 종로구 누하동 25-1',
 '서울시 서초구 양재동 1-28',
 '서울시 종로구 부암동 208-37',
 '종로구 당주동 5 로얄빌딩 지하 1층',
 '서울 마포구 연남동 257-8',
 '서울 중구 필동 1가 3-20',
 '서울 중구 북창동 93-36']

In [53]:
restaurant_list = []
for i in range(len(title_list)):
    info = (title_list[i], tel_list[i], address_list[i])
    restaurant_list.append(info)

In [54]:
restaurant_list

[('다성일식', '02-338-8951', '서울시 서대문구 창천동 72-36'),
 ('봉산집', '02-793-5022', '서울시 용산구 용산동3가 1-21'),
 ('창고43', '02-786-5959', '서울시 영등포구 여의도동 44-37 아일렉스타워15층(스카이점)'),
 ('돕감자탕', '02-499-2838', '서울시 광진구 화양동 9-22'),
 ('대보명가', '02-907-6998', '서울시 강북구 수유동 563-14'),
 ('해뜨는집', '02-764-6354', '서울시 성북구 동소문동 1가 62'),
 ('아이 해브어 드림', '02-3453-7696', '서울시 강남구 역삼동 821 이즈타워 B3'),
 ('아현동 간장게장', '02-312-7530', '서울시 마포구 아현동  282-1'),
 ('왕소금구이', '02-545-6844', '서울시 강남구 논현동 98-12'),
 ('라틀리에 모니크', '02-549-9210', '서울시 강남구 청담동 80-21'),
 ('비스트로 딩고', '02-518-7866', '서울시 강남구 신사동 526-11'),
 ('줄리에뜨', '02-535-4002', '서울시 서초구 반포4동 82-22'),
 ('충주집', '02-923-1068', '서울시 동대문구 제기동 138-2'),
 ('영화루', '02-738-1218', '서울시 종로구 누하동 25-1'),
 ('일품헌', '02-574-7117', '서울시 서초구 양재동 1-28'),
 ('립스테이크', '02-379-2581', '서울시 종로구 부암동 208-37'),
 ('오가와', '02-735-1001', '종로구 당주동 5 로얄빌딩 지하 1층'),
 ('까사디노아', '02-3142-1108', '서울 마포구 연남동 257-8'),
 ('충무로 주꾸미 불고기', '02-2279-0803', '서울 중구 필동 1가 3-20'),
 ('애성회관', '02-352-0303', '서울 중구 북창동 93-36')]